In [8]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display
import os

In [9]:
input_base_dir = 'BCS'

categories = ['belly pain', 'burping', 'cold_hot', 'discomfort', 'hungry', 'laugh', 'noise', 'silence', 'tired']

output_base_dir = 'BCS/spectrograms'

for category in categories:
    input_path = os.path.join(input_base_dir, category)
    output_path = os.path.join(output_base_dir, category)
    
    os.makedirs(output_path, exist_ok=True)
    
    if not os.path.exists(input_path):
        print(f"Warning: Input directory '{input_path}' does not exist")
        continue
    
    audio_files = [f for f in os.listdir(input_path) if f.endswith(('.wav', '.mp3', '.flac', '.ogg'))]
    
    if not audio_files:
        print(f"No audio files found in '{input_path}'")
        continue
    
    print(f"Processing {len(audio_files)} files from '{category}'...")
    
    for file in audio_files:
        filepath = os.path.join(input_path, file)
        
        try:
            y, sr = librosa.load(filepath, sr=22050)
            
            S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
            S_db = librosa.power_to_db(S, ref=np.max)
            
            plt.figure(figsize=(10, 4))
            librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='mel')
            plt.axis('off')
            
            output_filename = os.path.splitext(file)[0] + '.png'
            save_path = os.path.join(output_path, output_filename)
            plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
            plt.close()
            
        except Exception as e:
            print(f"Error processing {file}: {str(e)}")
    
    print(f"Completed processing '{category}'")

print("\nAll categories processed - original spectrograms created!")

Processing 124 files from 'belly pain'...
Completed processing 'belly pain'
Processing 108 files from 'burping'...
Completed processing 'burping'
Processing 108 files from 'cold_hot'...
Completed processing 'cold_hot'
Processing 135 files from 'discomfort'...
Completed processing 'discomfort'
Processing 382 files from 'hungry'...
Completed processing 'hungry'
Processing 108 files from 'laugh'...
Completed processing 'laugh'
Processing 108 files from 'noise'...
Completed processing 'noise'
Processing 108 files from 'silence'...
Completed processing 'silence'
Processing 132 files from 'tired'...
Completed processing 'tired'

All categories processed - original spectrograms created!


In [10]:
input_base_dir = 'BCS'

categories = ['belly pain', 'burping', 'cold_hot', 'discomfort', 'hungry', 'laugh', 'noise', 'silence', 'tired']

output_base_dir = 'BCS/spectrograms_filtered'

for category in categories:
    input_path = os.path.join(input_base_dir, category)
    output_path = os.path.join(output_base_dir, category)
    
    os.makedirs(output_path, exist_ok=True)
    
    if not os.path.exists(input_path):
        print(f"Warning: Input directory '{input_path}' does not exist")
        continue
    
    audio_files = [f for f in os.listdir(input_path) if f.endswith(('.wav', '.mp3', '.flac', '.ogg'))]
    
    if not audio_files:
        print(f"No audio files found in '{input_path}'")
        continue
    
    print(f"Processing {len(audio_files)} files from '{category}'...")
    
    for file in audio_files:
        filepath = os.path.join(input_path, file)
        
        try:
            y, sr = librosa.load(filepath, sr=22050)
            
            S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
            S_db = librosa.power_to_db(S, ref=np.max)
            
            threshold = np.mean(S_db) + np.std(S_db)
            mask = S_db > threshold
            
            freq_indices = np.where(np.sum(mask, axis=1) > 0)[0]
            time_indices = np.where(np.sum(mask, axis=0) > 0)[0]
            
            if len(freq_indices) == 0 or len(time_indices) == 0:
                print(f"  Skipping {file} - no significant content")
                continue
            
            S_filtered = S_db[freq_indices, :]
            S_filtered = S_filtered[:, time_indices]
            
            plt.figure(figsize=(10, 4))
            librosa.display.specshow(S_filtered, sr=sr, x_axis='time', y_axis='mel')
            plt.axis('off')
            
            output_filename = os.path.splitext(file)[0] + '.png'
            save_path = os.path.join(output_path, output_filename)
            plt.savefig(save_path, bbox_inches='tight', pad_inches=0)
            plt.close()
            
        except Exception as e:
            print(f"Error processing {file}: {str(e)}")
    
    print(f"Completed processing '{category}'")

print("\nAll categories processed with frequency filtering!")

Processing 124 files from 'belly pain'...
Completed processing 'belly pain'
Processing 108 files from 'burping'...
Completed processing 'burping'
Processing 108 files from 'cold_hot'...
Completed processing 'cold_hot'
Processing 135 files from 'discomfort'...
Completed processing 'discomfort'
Processing 382 files from 'hungry'...
Completed processing 'hungry'
Processing 108 files from 'laugh'...
Completed processing 'laugh'
Processing 108 files from 'noise'...
Completed processing 'noise'
Processing 108 files from 'silence'...
Completed processing 'silence'
Processing 132 files from 'tired'...
Completed processing 'tired'

All categories processed with frequency filtering!


In [11]:
import cv2
from sklearn.model_selection import train_test_split

spectrograms_dir = 'BCS/spectrograms'
categories = ['belly pain', 'burping', 'cold_hot', 'discomfort', 'hungry', 'laugh', 'noise', 'silence', 'tired']

X = []
y = []

print("Loading spectrogram images...")

for idx, category in enumerate(categories):
    category_path = os.path.join(spectrograms_dir, category)
    
    if not os.path.exists(category_path):
        print(f"Warning: Directory '{category_path}' does not exist")
        continue
    
    image_files = [f for f in os.listdir(category_path) if f.endswith('.png')]
    print(f"Loading {len(image_files)} images from '{category}'...")
    
    for image_file in image_files:
        image_path = os.path.join(category_path, image_file)
        
        try:
            img = cv2.imread(image_path)
            if img is not None:
                img_resized = cv2.resize(img, (224, 224))
                X.append(img_resized)
                y.append(idx)
        except Exception as e:
            print(f"Error loading {image_file}: {str(e)}")

X = np.array(X)
y = np.array(y)

print(f"\nBalancing dataset...")
class_counts = {}
for idx in range(len(categories)):
    class_counts[idx] = np.sum(y == idx)

min_samples = min(class_counts.values())

X_balanced = []
y_balanced = []

for idx in range(len(categories)):
    class_mask = y == idx
    class_indices = np.where(class_mask)[0]
    
    np.random.seed(42)
    selected_indices = np.random.choice(class_indices, min_samples, replace=False)
    
    for i in selected_indices:
        X_balanced.append(X[i])
        y_balanced.append(y[i])

X = np.array(X_balanced)
y = np.array(y_balanced)

print(f"\nDataset balanced!")
print(f"Total samples: {len(X)}")
print(f"Samples per class: {min_samples}")
print(f"\nClass distribution:")
for idx, category in enumerate(categories):
    count = np.sum(y == idx)
    print(f"  {category}: {count} samples")

print(f"\nNormalizing pixel values...")
X = X.astype('float32') / 255.0

print(f"\nSplitting into train and test sets...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nDataset Summary:")
print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")
print(f"\nTraining class distribution:")
for idx, category in enumerate(categories):
    count = np.sum(y_train == idx)
    print(f"  {category}: {count} samples")
print(f"\nTest class distribution:")
for idx, category in enumerate(categories):
    count = np.sum(y_test == idx)
    print(f"  {category}: {count} samples")

Loading spectrogram images...
Loading 124 images from 'belly pain'...
Loading 108 images from 'burping'...
Loading 108 images from 'cold_hot'...
Loading 135 images from 'discomfort'...
Loading 382 images from 'hungry'...
Loading 108 images from 'laugh'...
Loading 108 images from 'noise'...
Loading 108 images from 'silence'...
Loading 132 images from 'tired'...

Balancing dataset...

Dataset balanced!
Total samples: 972
Samples per class: 108

Class distribution:
  belly pain: 108 samples
  burping: 108 samples
  cold_hot: 108 samples
  discomfort: 108 samples
  hungry: 108 samples
  laugh: 108 samples
  noise: 108 samples
  silence: 108 samples
  tired: 108 samples

Normalizing pixel values...

Splitting into train and test sets...

Dataset Summary:
Training samples: 777
Test samples: 195

Training class distribution:
  belly pain: 86 samples
  burping: 87 samples
  cold_hot: 86 samples
  discomfort: 86 samples
  hungry: 86 samples
  laugh: 87 samples
  noise: 86 samples
  silence: 87 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import seaborn as sns

print("Building CNN Model...")

model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),
    
    layers.Conv2D(256, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),
    
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.4),
    
    layers.Dense(len(categories), activation='softmax')
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

print("\nTraining CNN model...")

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=15,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("\nModel training completed!")

print("\nEvaluating model on training set...")
train_loss, train_accuracy = model.evaluate(X_train, y_train, verbose=0)
print(f"Train Accuracy: {train_accuracy:.4f} ({train_accuracy*100:.2f}%)")

print("\nEvaluating model on test set...")
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")

print(f"Overfitting Gap: {(train_accuracy - test_accuracy)*100:.2f}%")

y_pred_probs = model.predict(X_test, verbose=0)
y_pred = np.argmax(y_pred_probs, axis=1)

print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=categories))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=categories, yticklabels=categories)
plt.title(f'Confusion Matrix - CNN Model\nTrain Accuracy: {train_accuracy:.2%} | Test Accuracy: {test_accuracy:.2%} | Gap: {(train_accuracy - test_accuracy)*100:.1f}%')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

print("\nPer-class Accuracy:")
for idx, category in enumerate(categories):
    class_mask = y_test == idx
    if np.sum(class_mask) > 0:
        class_accuracy = accuracy_score(y_test[class_mask], y_pred[class_mask])
        print(f"  {category}: {class_accuracy:.4f} ({class_accuracy*100:.2f}%)")

plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Test Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Test Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

Building CNN Model...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 batch_normalization_6 (Batc  (None, 222, 222, 32)     128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 111, 111, 32)     0         
 2D)                                                             
                                                                 
 dropout_6 (Dropout)         (None, 111, 111, 32)      0         
                                                                 
 conv2d_5 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 batch_normalization_7 (Batc  (N